<a href="https://colab.research.google.com/github/kirath2205/Final-Year-Project/blob/main/defensive_distillation_resnet_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, UpSampling2D, Flatten, BatchNormalization, Dense, Dropout, GlobalAveragePooling2D , UpSampling3D , Lambda
from tensorflow.keras import optimizers
from keras.datasets import cifar100,cifar10,fashion_mnist
import tensorflow as tf
from keras.utils import np_utils
import matplotlib.pyplot as plt
import time
from skimage.transform import resize
from keras import applications
from tensorflow.keras.applications.resnet50 import ResNet50 , preprocess_input , decode_predictions
from keras.preprocessing.image import ImageDataGenerator

In [79]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.2,
        temperature=20,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

    def call(self, inputs, *args, **kwargs):
       return self.student(inputs)

In [80]:
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

In [81]:
def select_dataset(index=1): #1 for cifar10 , 2 for cifar100 , 3 for fashion mnist
  if(index == 1):
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    channel = 3
    num_classes = 10
  if(index == 2):
    (x_train, y_train), (x_test, y_test) = cifar100.load_data()
    num_classes = 100
    channel = 3
  if(index == 3):
    (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
    x_train = x_train.reshape((60000, 28, 28, 1))
    x_test =  x_test.reshape((10000, 28, 28, 1))
    num_classes = 10
    y_train = np_utils.to_categorical(y_train, num_classes)
    y_test = np_utils.to_categorical(y_test, num_classes)
    channel = 1
    return (x_train , y_train , x_test , y_test , num_classes , channel)

  #Pre-process the data
  x_train = preprocess_input(x_train)
  x_test = preprocess_input(x_test)

  datagen = ImageDataGenerator(preprocessing_function=get_random_eraser(v_l=0, v_h=1, pixel_level=True))
  datagen.fit(x_train)
  y_train = np_utils.to_categorical(y_train, num_classes)
  y_test = np_utils.to_categorical(y_test, num_classes)

  return (x_train , y_train , x_test , y_test , num_classes , channel , datagen)

In [82]:
index = 1
x_train , y_train , x_test , y_test , num_classes ,channel , datagen = select_dataset(index)
model_name = 'desktop/Trained_models/defensive_distillation_keras_resnet50_cifar10'
model_path = 'desktop/Trained_models/defensive_distillation_keras_resnet50_cifar10.h5'

In [ ]:
index = 2
x_train , y_train , x_test , y_test , num_classes ,channel , datagen = select_dataset(index)
model = Resnet50(num_classes,channel)
model_name = 'desktop/Trained_models/defensive_distillation_keras_resnet50_cifar100'
model_path = 'desktop/Trained_models/defensive_distillation_keras_resnet50_cifar100.h5'

In [83]:
temp = 200
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in resnet_model.layers:
    if isinstance(layer, BatchNormalization):
      layer.trainable = True
    else:
      layer.trainable = False
teacher = keras.Sequential(
    [
        layers.UpSampling2D((7,7)),
        resnet_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256),
        layers.ReLU(),
        layers.Dropout(0.25),
        layers.BatchNormalization(),
        layers.Dense(32),
        layers.ReLU(),
        layers.Dropout(0.1),
        layers.BatchNormalization(),
        layers.Dense(num_classes),
        layers.Lambda((lambda x: x / temp)),
    ],
    name="teacher",
)

# Create the student
student = keras.Sequential(
    [
        layers.UpSampling2D((7,7)),
        resnet_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256),
        layers.ReLU(),
        layers.Dropout(0.25),
        layers.BatchNormalization(),
        layers.Dense(32),
        layers.ReLU(),
        layers.Dropout(0.1),
        layers.BatchNormalization(),
        layers.Dense(num_classes),
        layers.Lambda((lambda x: x / temp)),
    ],
    name="student",
)


In [84]:
epochs = 40
callbacks = [ 
    tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau( factor = 0.7, patience = 5, min_lr = 0.000001, verbose = 1 ), #patience = 7 and 20 for cifar-100 , patience = 5 and 10 for cifar-10
    #tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy' , patience = 10)
  ]
batch_size = 128
teacher.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.CategoricalAccuracy()],
)

# Train and evaluate teacher on data.
teacher.fit_generator(datagen.flow(x_train, y_train,
                                  batch_size=batch_size),
                                  steps_per_epoch=x_train.shape[0] // batch_size,
                                  epochs=epochs,
                                  validation_data=(x_test, y_test))
teacher.evaluate(x_test, y_test)

Epoch 1/40
390/390 [==============================] - 165s 423ms/step - loss: 2.2475 - categorical_accuracy: 0.7730 - val_loss: 2.1710 - val_categorical_accuracy: 0.9027
Epoch 2/40
390/390 [==============================] - 166s 426ms/step - loss: 2.1114 - categorical_accuracy: 0.8909 - val_loss: 2.0177 - val_categorical_accuracy: 0.9282
Epoch 3/40
390/390 [==============================] - 167s 428ms/step - loss: 1.9085 - categorical_accuracy: 0.9163 - val_loss: 1.7841 - val_categorical_accuracy: 0.9404
Epoch 4/40
390/390 [==============================] - 167s 427ms/step - loss: 1.6590 - categorical_accuracy: 0.9261 - val_loss: 1.5065 - val_categorical_accuracy: 0.9396
Epoch 5/40
390/390 [==============================] - 166s 426ms/step - loss: 1.3901 - categorical_accuracy: 0.9361 - val_loss: 1.2248 - val_categorical_accuracy: 0.9422
Epoch 6/40
390/390 [==============================] - 166s 427ms/step - loss: 1.1284 - categorical_accuracy: 0.9411 - val_loss: 0.9909 - val_categoric

[0.16183745861053467, 0.9526000022888184]

In [85]:
distiller = Distiller(student=student, teacher=teacher)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.CategoricalAccuracy()],
    student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.2,
    temperature=200,
)

# Distill teacher to student
distiller.fit_generator(datagen.flow(x_train, y_train,
                                  batch_size=batch_size),
                                  steps_per_epoch=x_train.shape[0] // batch_size,
                                  epochs=epochs,
                                  validation_data=(x_test, y_test),
                                  callbacks = callbacks)

# Evaluate student on test dataset
distiller.evaluate(x_test, y_test)

Epoch 1/40
390/390 [==============================] - 227s 583ms/step - categorical_accuracy: 0.9300 - student_loss: 2.2400 - distillation_loss: 1.1957e-04 - val_categorical_accuracy: 0.9400 - val_student_loss: 2.1615 - lr: 0.0010
Epoch 2/40
390/390 [==============================] - 227s 581ms/step - categorical_accuracy: 0.9542 - student_loss: 2.1033 - distillation_loss: 1.1784e-04 - val_categorical_accuracy: 0.9547 - val_student_loss: 2.0042 - lr: 0.0010
Epoch 3/40
390/390 [==============================] - 227s 581ms/step - categorical_accuracy: 0.9583 - student_loss: 1.8964 - distillation_loss: 1.1284e-04 - val_categorical_accuracy: 0.9456 - val_student_loss: 1.7503 - lr: 0.0010
Epoch 4/40
390/390 [==============================] - 226s 580ms/step - categorical_accuracy: 0.9644 - student_loss: 1.6386 - distillation_loss: 1.0553e-04 - val_categorical_accuracy: 0.9556 - val_student_loss: 1.4695 - lr: 0.0010
Epoch 5/40
390/390 [==============================] - 226s 581ms/step - cate

0.9577999711036682

In [86]:


def make_prediction(model , image , true_value):
  true_label_index = -1
  for i in range(len(true_value)):

    if(true_value[i]==1):
      true_label_index = i
      break

  prediction = model.predict(image)[0]
  probability = float('-inf')
  predicted_label_index = -1

  for i in range(len(prediction)):

    if(prediction[i]>probability):
      probability = prediction[i]
      predicted_label_index = i

  if(true_label_index!=predicted_label_index):
    return 1
  return 0



In [87]:
import keras,os
import tensorflow as tf
from keras.models import Sequential,Model
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D , Dropout
import numpy as np
from tensorflow import keras
from keras.initializers import glorot_uniform
from keras import applications
from tensorflow.keras.applications.resnet50 import ResNet50 , preprocess_input
from keras import regularizers
from absl import app, flags

# Install bleeding edge version of cleverhans
!pip install git+https://github.com/tensorflow/cleverhans.git#egg=cleverhans
from cleverhans.tf2.attacks.projected_gradient_descent import projected_gradient_descent
from cleverhans.tf2.attacks.fast_gradient_method import fast_gradient_method

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [88]:
from keras.preprocessing.image import img_to_array

epsilon_end = 0.8
#logits_model = tf.keras.Model(model.input,model.layers[-1].output)
total_images = 200

result={}
epsilon = 0.1
while(epsilon <= epsilon_end):

  fgsm_counter = 0
  pgd_counter = 0
  print("Epsilon value - ",epsilon)
  print()
  for image_index in range(total_images):
  
  
    image = x_test[image_index]
    image = img_to_array(image)
    image = image.reshape(1, 32, 32, 3)
    true_value = y_test[image_index]
    original_prediction = make_prediction(distiller,image,true_value)
    fgsm_sample = fast_gradient_method(distiller, image, epsilon, np.inf, targeted=False)
    pgd_sample = projected_gradient_descent(distiller, image, epsilon, 0.01, 40, np.inf)
    print(epsilon,' ',image_index)
    fgsm_prediction = make_prediction(distiller , fgsm_sample , true_value)
    pgd_prediction = make_prediction(distiller , pgd_sample , true_value)
    pgd_counter+=pgd_prediction
    fgsm_counter+=fgsm_prediction
    print(pgd_counter,' ',fgsm_counter)
  result[epsilon] = (fgsm_counter , pgd_counter)
  epsilon+=0.1


Epsilon value -  0.1

0.1   0
0   0
0.1   1
0   0
0.1   2
0   0
0.1   3
0   0
0.1   4
0   0
0.1   5
0   0
0.1   6
0   0
0.1   7
0   0
0.1   8
0   0
0.1   9
0   0
0.1   10
0   0
0.1   11
0   0
0.1   12
0   0
0.1   13
0   0
0.1   14
0   0
0.1   15
0   0
0.1   16
0   0
0.1   17
0   0
0.1   18
0   0
0.1   19
0   0
0.1   20
0   0
0.1   21
0   0
0.1   22
0   0
0.1   23
0   0
0.1   24
0   0
0.1   25
0   0
0.1   26
0   0
0.1   27
0   0
0.1   28
0   0
0.1   29
0   0
0.1   30
0   0
0.1   31
0   0
0.1   32
0   0
0.1   33
0   0
0.1   34
0   0
0.1   35
0   0
0.1   36
0   0
0.1   37
0   0
0.1   38
0   0
0.1   39
0   0
0.1   40
0   0
0.1   41
0   0
0.1   42
0   0
0.1   43
0   0
0.1   44
0   0
0.1   45
0   0
0.1   46
0   0
0.1   47
0   0
0.1   48
0   0
0.1   49
0   0
0.1   50
0   0
0.1   51
0   0
0.1   52
0   0
0.1   53
0   0
0.1   54
0   0
0.1   55
0   0
0.1   56
0   0
0.1   57
1   1
0.1   58
1   1
0.1   59
2   2
0.1   60
2   2
0.1   61
2   3
0.1   62
2   3
0.1   63
2   3
0.1   64
2   3
0.1   65
2   

In [89]:
for key in result.keys():
  print(key,' ',result[key])

0.1   (15, 19)
0.2   (28, 41)
0.30000000000000004   (41, 64)
0.4   (54, 79)
0.5   (61, 79)
0.6   (68, 79)
0.7   (75, 79)
0.7999999999999999   (79, 79)
